#This notebook was run in colab. 
Specs: HighRam, T4 GPU

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
from transformers import GPTNeoXForCausalLM, AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5")
# Note: This model takes 15 GB of Vram when loaded in 8bit
model = GPTNeoXForCausalLM.from_pretrained(
  "OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5",device_map="auto", load_in_8bit=True)
# for cpu ver
# model = AutoModelForCausalLM.from_pretrained("OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5", torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
#run once to initialize and load model
message = "<|prompter|>What is a meme, and what's the history behind this word?<|endoftext|><|assistant|>"
inputs = tokenizer(message, return_tensors="pt").to(model.device)
tokens = model.generate(**inputs,  max_new_tokens=1000, do_sample=True, temperature=0.8)
response = tokenizer.decode(tokens[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [14]:
def ngenerate(input):
  input_ids = tokenizer(input, return_tensors="pt").to(model.device)
  # with torch.cuda.amp.autocast():
  output = model.generate(input_ids, max_new_tokens=1000, do_sample=True, eos_token_id=model.config.eos_token_id, early_stopping=True, num_return_sequences=1)
  output = output.cpu()
  output_text = tokenizer.decode(output[0], skip_special_tokens=False)
PROMPTER_START_TOKEN = '<|prompter|>'
ASSISTANT_START_TOKEN = '<|assistant|>'
END_TOKEN = '<|endoftext|>'
def ntalk(new_message_raw, prev_message):
  new_message_w_token = f'{PROMPTER_START_TOKEN}{new_message_raw}{END_TOKEN}{ASSISTANT_START_TOKEN}'
  # print(new_message_w_token)
  message_input = prev_message + new_message_w_token
  response = ngenerate(message_input)
  just_response = response.split(ASSISTANT_START_TOKEN)[-1].replace(END_TOKEN, "")
  return just_response, response

In [15]:
%%time
tracked_convo = ""
new_message = "Hello there!"
pure_response, tracked_convo = ntalk(new_message, tracked_convo)
print(pure_response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:249 in           │
│ __getattr__                                                                                      │
│                                                                                                  │
│    246 │                                                                                         │
│    247 │   def __getattr__(self, item: str):                                                     │
│    248 │   │   try:                                                                              │
│ ❱  249 │   │   │   return self.data[item]                                                        │
│    250 │   │   except KeyError:                                                                  │
│    251 │   │   │   raise AttributeError                                                          │
│    252                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'shape'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1335 in time            │
│                                                                                                  │
│   1332 │   │   else:                                                                             │
│   1333 │   │   │   st = clock2()                                                                 │
│   1334 │   │   │   try:                                                                          │
│ ❱ 1335 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1336 │   │   │   │   out=None                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:3                                                                                    │
│ in ntalk:14                                                                                      │
│ in ngenerate:4                                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1293 in generate        │
│                                                                                                  │
│   1290 │   │   inputs_tensor, model_input_name, model_kwargs = self._pre

In [16]:
def generate(message):
  inputs = tokenizer(message, return_tensors="pt").to(model.device)
  tokens = model.generate(**inputs,  max_new_tokens=1000, do_sample=True, temperature=0.8, early_stopping=True, num_return_sequences=1)
  response = tokenizer.decode(tokens[0])
  return response
PROMPTER_START_TOKEN = '<|prompter|>'
ASSISTANT_START_TOKEN = '<|assistant|>'
END_TOKEN = '<|endoftext|>'
def talk(new_message_raw, prev_message):
  new_message_w_token = f'{PROMPTER_START_TOKEN}{new_message_raw}{END_TOKEN}{ASSISTANT_START_TOKEN}'
  # print(new_message_w_token)
  message_input = prev_message + new_message_w_token
  response = generate(message_input)
  just_response = response.split(ASSISTANT_START_TOKEN)[-1].replace(END_TOKEN, "")
  return just_response, response

In [17]:
%%time
tracked_convo = ""
new_message = "Hello there!"
pure_response, tracked_convo = talk(new_message, tracked_convo)
print(pure_response)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello! How can I help you today?
CPU times: user 2.07 s, sys: 3.35 ms, total: 2.07 s
Wall time: 2.07 s


In [18]:
%%time
new_message = "What is your name?"
pure_response, tracked_convo = talk(new_message, tracked_convo)
print(pure_response)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


I am Open Assistant, a chat based assistant that can understand natural language and generate responses.
CPU times: user 4.62 s, sys: 4.32 ms, total: 4.63 s
Wall time: 4.63 s


In [21]:
%%time
new_message = "Can you make a description for a shoe? Make it similar to this: '• Heavyweight denim jeans • Custom butter tag on back pocket & coin pocket • Baggy fit • Built in drawcord waist'"
pure_response, tracked_convo = talk(new_message, tracked_convo)
print(pure_response)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


The shoe is a comfortable, stylish, and highly practical piece of footwear that can be worn by people of all ages and backgrounds. It is made from durable materials and is designed to provide great support and comfort. The shoe has a classic styling, with a low-profile design and a soft, supple leather upper. The sole is made from a durable, shock-absorbent material and is designed to provide excellent traction on a variety of surfaces. Whether you're on your feet all day at work or out for a night on the town, the shoe will help you stay comfortable and stylish. It is an excellent choice for both casual and dressy occasions.
CPU times: user 24.9 s, sys: 33.5 ms, total: 25 s
Wall time: 24.9 s


In [20]:
pure_response

"The shoe is a comfortable, stylish, and highly practical piece of footwear that can be worn by people of all ages and backgrounds. It is made from durable materials and is designed to provide great support and comfort. The shoe has a classic styling, with a low-profile design and a soft, supple leather upper. The sole is made from a durable, shock-absorbent material and is designed to provide excellent traction on a variety of surfaces. Whether you're on your feet all day at work or out for a night on the town, the shoe will help you stay comfortable and stylish. It is an excellent choice for both casual and dressy occasions."

In [22]:
pure_response

"The shoe is a comfortable, stylish, and highly practical piece of footwear that can be worn by people of all ages and backgrounds. It is made from durable materials and is designed to provide great support and comfort. The shoe has a classic styling, with a low-profile design and a soft, supple leather upper. The sole is made from a durable, shock-absorbent material and is designed to provide excellent traction on a variety of surfaces. Whether you're on your feet all day at work or out for a night on the town, the shoe will help you stay comfortable and stylish. It is an excellent choice for both casual and dressy occasions."

In [23]:
tracked_convo

"<|prompter|>Hello there!<|endoftext|><|assistant|>Hello! How can I help you today?<|endoftext|><|prompter|>What is your name?<|endoftext|><|assistant|>I am Open Assistant, a chat based assistant that can understand natural language and generate responses.<|endoftext|><|prompter|>Can you make a description for a shoe?<|endoftext|><|assistant|>The shoe is a comfortable, stylish, and highly practical piece of footwear that can be worn by people of all ages and backgrounds. It is made from durable materials and is designed to provide great support and comfort. The shoe has a classic styling, with a low-profile design and a soft, supple leather upper. The sole is made from a durable, shock-absorbent material and is designed to provide excellent traction on a variety of surfaces. Whether you're on your feet all day at work or out for a night on the town, the shoe will help you stay comfortable and stylish. It is an excellent choice for both casual and dressy occasions.<|endoftext|><|prompte

In [26]:
tracked_convo = ""

In [27]:
%%time

new_message = "Make a description about a green shirt."
pure_response, tracked_convo = talk(new_message, tracked_convo)
print(pure_response)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


This green shirt was made from soft, lightweight fabric that feels comfortable to wear. The sleeves are long enough to keep arms cool, and the subtle pattern adds a pop of color. The material is breathable, so it is perfect for warm weather.
CPU times: user 9.65 s, sys: 18.4 ms, total: 9.67 s
Wall time: 9.65 s


In [28]:
%%time

new_message = "Convert that description into bullet points"
pure_response, tracked_convo = talk(new_message, tracked_convo)
print(pure_response)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Bullet points:

- Green fabric
- Long sleeves
- Breathable
- Pattern adds color
- Comfortable to wear
- Great for warm weather
CPU times: user 7.18 s, sys: 8.82 ms, total: 7.18 s
Wall time: 7.18 s
